In [1]:
#!pip install langchain
#!pip install langchain_community
#!pip install chromadb
#!pip install langchain-openai

In [1]:
import pandas as pd
import certifi
import urllib3
import warnings
import os
import openai

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

import pyface
import importlib_resources
from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
#KB 이벤트 웹데이터
web_data = pd.read_excel('/Users/parkjungeun/DML_crawling/KB_공모전/df_total.xlsx')
result = [x for x in web_data['알아두세요']]
#docs = ' '.join(result)
docs = result
print(docs)

전체 이벤트 기간 중 1회만 당첨 가능하며, 이전 회차에 당첨된 이력이 있는 고객님은 중복 당첨이 불가합니다.경품은 KB국민은행에 등록된 연락처 기준으로 발송되오니 등록된 고객정보를 반드시 확인하여주시기 바랍니다.(연락처 불일치 시 수령 불가할 수 있습니다.) 내 고객정보 바로가기 >경품을 지급하기 위해 개인(신용)정보 수집·이용·제공 동의가 필수입니다.KB국민은행 고객정보에 등록된 핸드폰번호로 최초 1회 발송 후 연락처 오류로 경품 수령이 불가 시 당첨이 무효처리 됩니다. (재배송 및 대체 경품으로의 변경 불가)모바일쿠폰은 유효기간 내 사용가능하며 발송된 쿠폰의 유효기간이 경과한 경우 재발송이 불가합니다.경품 제공을 위해 수탁 업체에 고객님의 정보를 제공하고 경품 발송 업무를 위탁합니다. (모바일 쿠폰 발송 수탁자 : ㈜ 쿠프마케팅, 제공범위: 성명/휴대전화번호/CI번호, 대상업무 : 모바일상품권 발송)이벤트 추첨은 난수 입력을 통한 자동 추첨방식으로 진행됩니다.경품 당첨자는 별도 유선통보하지 않으며, KB스타뱅킹 앱 > 혜택 > 이벤트 > 응모/당첨확인 > 당첨 이벤트 보기 > 해당 이벤트 클릭 에서 확인 가능합니다.본 이벤트 수령 금액을 포함하여 기타소득이 연간 3백만원을 초과할 경우 종합소득 신고대상에 해당됩니다.본 이벤트는 KB국민은행의 사정으로 변경 또는 중단될 수 있으며, 이벤트 경품은 사정상 동일 금액대의 다른 상품으로 변경될 수 있습니다.일반금융소비자는 은행이 계약체결을 권유하는 경우 및 일반금융소비자가 설명을 요청하는 경우에도 중요한 사항을 이해할 수 있도록 설명 받을 권리가 있습니다.본 광고물은 이벤트 관련 홍보물이며 서비스 이용 전 자세한 사항에 대해 서비스 설명서 및 약관을 읽어보시기 바랍니다.기타 자세한 이벤트 내용은 KB국민은행 고객센터(☎1588-9999) 및 KB국민은행 홈페이지(www.kbstar.com) > 전체서비스 > 생활/혜택 > 이벤트를 참조하시기 바랍니다.상품(서비스) 가입 전 상품(서비스)설명서 및 약관을 읽어보시기

In [8]:
#텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splited_docs = text_splitter.create_documents(docs) #.split_documents
print(len(splited_docs))
print(splited_docs[10].page_content)

1334
따라 예금보호 대상 금융상품으로 운용되는 적립금에 대하여 다른 보호상품과는 별도로 1인당 “5천만원까지”(운용되는 금융상품 판매회사별 보호상품 합산) 보호됩니다.이 투자자는 퇴직연금에 대하여 은행으로부터 충분한 설명을 받을 권리가 있으며, 가입 전 상품설명서 및 약관을 반드시 읽어보시기 바랍니다.과세기준 및 과세방법은 향후 세법개정 등에 따라 변동될 수 있습니다.연금수령요건을 갖추지 못하고 중도해지하거나 계약기간 종료 후 연금 이외의 형태로 수령하는 경우 세액공제 받은 납입원금 및 수익에 대해 기타소득세(16.5%)세율이 부과될 수 있습니다.DC(사용자부담금) 연0.38%~0.7%, DC(가입자부담금) 수수료 연0.40%보수(수수료)는 상품별로 다르게 부과될 수 있습니다.디폴트옵션 포트폴리오는 단순히 여러 상품을 한번에 가입하는 바구니의 개념으로, 개별 상품의 보수는 발생하나 포트폴리오 구성에 따른 추가적인 비용은 발생하지 않습니다. 그렇기 때문에 상품에 대한 혜택과 불이익은 상품 내 담기는 구성상품에 영향을 받으므로, 피투자상품에 대한 내용을 별도로 숙지할 필요가 있습니다.본 포트폴리오는 금융시장 상황 및 각 집합투자재산의 가치변동 등을 고려하여 자산배분 비중의 조정이 필요하다고 판단되는 경우 고용노동부장관에게 승인신청 후 최초 설정한 비중을 변경할 수 있습니다. 판매회사가 포트폴리오의 비중을 변경하려는 경우 14일 이내에 가입자에게 해당 사실을 통보하고, 가입자의 적립금을 변경된 비중으로 운용할 수 있으며, 이를 희망하지 않는 가입자는 스스로 다른 운용방법을 선택할 수 있습니다.기타 자세한 이벤트 내용은 영업점 직원 또는 퇴직연금 전용 상담센터(☎1599-0099)/고객센터(☎1588-9999)로 문의하시거나 KB국민은행 홈페이지(www.kbstar.com)를 참조하시기 바랍니다.※ 한국금융투자협회 심사필 제24-01155호 (2024-03-29 ~ 2024-05-31)※ KB국민은행 준법감시인 심의필 제2024-1422-1호(2024.03.28)

In [9]:
os.environ["OPENAI_API_KEY"] = "sk-proj-AO3KelC1emoiTrtYed3jT3BlbkFJVuDMTmSkpm2J4S7me2nO"

#임베딩
vectorstore = Chroma.from_documents(documents = splited_docs, embedding=OpenAIEmbeddings())

/opt/homebrew/Caskroom/miniforge/base/envs/CW/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [10]:
prompt_template = '''
Answer the question base on the following contxt:
{context}

question : {question}
'''

prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

retriever = vectorstore.as_retriever()

In [11]:
# Combine Documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

#Chain 연결
chain = (
    {'context' : retriever | format_docs, 'question':RunnablePassthrough()}
     | prompt
     | model
     | StrOutputParser()
)

#answer = chain.invoke("KB이벤트 내에서 '집합투자상품'와 같은 어려운 금융 용어를 10개 뽑아주고 각 용어에 대해 해석한 결과를 보여줘.")
answer = chain.invoke("KB이벤트 내에서 '집합투자상품','신탁계약','예금자보호법'과 같은 어려운 금융 용어를 15개 뽑아주고 각 용어에 대한 결과를 단어 key, 해석 value json형식으로 보여줘.")
answer

'{\n  "집합투자상품": "여러 투자자들이 자금을 모아 운용하는 상품으로, 운용 결과에 따라 손실이 발생할 수 있음",\n  "신탁계약": "자산을 신탁사에 맡겨 신탁사가 대신 운용하는 계약",\n  "예금자보호법": "예금자의 이자 및 원금을 보호하기 위한 법률",\n  "투자원금": "투자에 사용된 자본",\n  "환매방법": "투자상품을 되돌려 받는 방법",\n  "투자설명서": "투자상품의 세부 내용을 설명하는 문서",\n  "환율변동": "통화 가치의 변동으로 인한 위험",\n  "펀드": "여러 투자자들의 자금을 모아 운용하는 투자상품",\n  "원금 손실": "투자한 원금을 잃는 것",\n  "국가의 시장": "해당 국가의 경제적 활동을 나타내는 시장",\n  "정치 및 경제상황": "해당 국가의 정치적 및 경제적 상황",\n  "위험": "손실이 발생할 가능성이 있는 상황",\n  "수탁업체": "외부 업체에 일부 업무를 위탁하는 기업",\n  "연락불가": "연락이 되지 않는 상태",\n  "고객정보": "고객에 대한 정보"\n}'

In [12]:
file_path = '/Users/parkjungeun/Downloads/output.txt'

with open(file_path, "w") as file:
    file.write(answer)